In [268]:
import sys
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import mysql.connector
db_name = input()
db_input = input("input you database : ")
mydb = mysql.connector.connect(
    host="43.225.141.48",
    port = "3306",
    user=db_name,
    password=db_input,
    database="tiktrack_beta"
)
year = '2023'
print( f"Python {sys.version}\nPandas {pd.__version__}\nNumPy {np.__version__}") 
df = pd.read_sql(f'''select 
t1.*
-- , sum((t1.lev_data_d_day*24)*60) as x
, concat(t2.firstname, ' ', t2.lastname) as full_name
, t5.leave_type_name
, t3.dep_name, t4.brancheName 
, t2.`startworkdate`,t2.`quitworkdate`
from leave_data as t1
left join users as t2 on t1.employee_id = t2.employee_id
left join department as t3 on t2.dep_id = t3.dep_id
left join branches as t4 on t2.`branche_id` = t4.`branche_id`
left join ms_leave_type as t5 on t1.leave_type = t5.`leave_type_code` and t2.company_id = t5.company_id
where YEAR(t1.`lev_data_date_start`) = {year}
and t2.company_id in ('1','7','8','9')
and t1.employee_id in ('2018071216000124','2018071216000254','2018071216000256')
and t1.lev_approve_status = '1'
and t5.status = '1'
-- GROUP BY t1.leave_type, t1.employee_id
order by t3.dep_name asc''', mydb)

# Display the first few rows of the dataframe
print(df.head())
# Load the pokemon dataset

# df = pd.read_csv(year+'.csv')  

Python 3.9.12 (main, Jun  1 2022, 06:34:44) 
[Clang 12.0.0 ]
Pandas 1.5.2
NumPy 1.22.4
   lev_data_id lev_data_date_start   lev_data_date_end  lev_data_d_day  \
0        32240 2023-01-20 08:00:00 2023-01-20 17:00:00             1.0   
1        32761 2023-02-09 16:00:00 2023-02-09 17:00:00             0.0   
2        32940 2023-02-15 14:00:00 2023-02-15 17:00:00             0.0   
3        33252 2023-02-24 14:00:00 2023-02-24 17:00:00             0.0   
4        31751 2023-01-05 13:00:00 2023-01-05 17:00:00             0.0   

   lev_data_d_hour  lev_data_d_minute  leave_type     detail  \
0              0.0                0.0          35    พักร้อน   
1              1.0                0.0          35  ลาพักร้อน   
2              3.0                0.0          35    พักร้อน   
3              3.0                0.0          35  ลาพักร้อน   
4              4.0                0.0          35    พักร้อน   

   lev_approve_status       employee_id  ...            updateAt  \
0              

/var/folders/kg/w853qbnj03x0b8wb35_f_v540000gp/T/ipykernel_37543/3642492933.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'''select


In [258]:
# Cleansing
# df['startworkdate'] = df['startworkdate'].replace(['0000-00-00'], '2017-03-01')
df['total_leave_data_minute'] = df['lev_data_d_day'] * 480 + df['lev_data_d_hour'] * 60 + df['lev_data_d_minute']
df['lev_data_date_start'] = pd.to_datetime(df['lev_data_date_start'])
df['year'] = df.loc[df['lev_data_date_start'].notnull(), 'lev_data_date_start'].dt.year
df['lev_data_date_start'] = pd.to_datetime(df['lev_data_date_start'])

In [259]:
def calculate_leave_allowance(duration):
    if duration == 0:
        return 0
    elif duration < 3:
        return 6
    elif duration < 5:
        return 9
    elif duration < 7:
        return 12
    else:
        return 15

def assign_leave_allowance(row):
    leave_type = row['leave_type_name']
    duration = row['durationwork_years']
    
    if leave_type == 'ลาป่วย (ไม่หักค่าจ้าง)':
        return 30
    elif leave_type == 'ลากิจ (หักค่าจ้าง)':
        return 365
    elif leave_type == 'ลากิจ - เพื่อจัดงานแต่ง':
        return 3
    elif leave_type == 'ลาคลอด (ไม่หักค่าจ้าง)':
        return 45
    elif leave_type == 'ลากิจ - เพื่ออุปสมบท (กรณีอายุงานถึง 3 ปี)':
        return 15
    elif leave_type == 'ลาคลอด หักค่าจ้าง':
        return 53
    elif leave_type == 'ลาพักร้อน':
        return calculate_leave_allowance(duration)
    elif leave_type == 'ลากิจ - เพื่ออุปสมบท (กรณีอายุงานไม่ถึง 3 ปี)':
        return 15
    elif leave_type == 'ลากิจ (ไม่หักค่าจ้าง)':
        return 3
    elif leave_type == 'ลาป่วย (หักค่าจ้าง)':
        return 365
    elif leave_type == 'ลากิจเพื่อจัดงานศพ ไม่หักค่าจ้าง':
        return 7

def leave_cal_day_result(row):
    leave_allow = row['leave_allow']
    leave_total = row['total_leave_data_minute']
    if leave_allow * 480 >= leave_total:
        return ((leave_allow * 480) - leave_total) // 480
    else :
        return -((leave_total -(leave_allow * 480) ) // 480)
    
def leave_cal_hour_result(row):
    leave_allow = row['leave_allow']
    leave_total = row['total_leave_data_minute']
    leave_d_result = row['leave_day_enough']
    if leave_allow * 480 >= leave_total:
        return (((leave_allow * 480) - leave_total) - leave_d_result * 480) // 60
    else :
        return -(((leave_total -(leave_allow * 480)) - abs(leave_d_result) * 480) // 60)

def leave_cal_minute_result(row):
    leave_allow = row['leave_allow']
    leave_total = row['total_leave_data_minute']
    leave_d_result = row['leave_day_enough']
    leave_h_result = row['leave_hours_enough']
    if leave_allow * 480 >= leave_total:
        return (((leave_allow * 480) - leave_total) - leave_d_result * 480) - leave_h_result *60
    else :
        return -(((leave_total - (leave_allow * 480)) - abs(leave_d_result) * 480) - abs(leave_h_result) *60)

summed_df = df.groupby(['employee_id', 'full_name', 'leave_type_name','year','startworkdate']).sum().reset_index().sort_values(by='full_name')
summed_df['leave_day'] = summed_df['total_leave_data_minute'] // 480
summed_df['leave_hours'] = (summed_df['total_leave_data_minute'] - summed_df['leave_day']*480)//60
summed_df['leave_minutes'] = summed_df['total_leave_data_minute'] - summed_df['leave_day']*480 - summed_df['leave_hours']*60
summed_df['leave_used'] = summed_df.apply(lambda x: str(x['leave_day']) + ':' + str(x['leave_hours']) + ':' + str(x['leave_minutes']), axis=1)
summed_df['end_of_year'] = pd.to_datetime(summed_df['year'], format='%Y') + pd.offsets.YearEnd(0)
summed_df['startworkdate'] = pd.to_datetime(summed_df['startworkdate'], format='%Y-%m-%d')
summed_df['durationwork'] = summed_df['end_of_year'] - summed_df['startworkdate']
summed_df['durationwork_years'] = summed_df['durationwork'].dt.days // 365.25

#add Funciton allow
summed_df['leave_allow'] = summed_df.apply(assign_leave_allowance, axis=1)
summed_df['leave_day_enough'] = summed_df.apply(leave_cal_day_result, axis=1)
summed_df['leave_hours_enough'] = summed_df.apply(leave_cal_hour_result, axis=1)
summed_df['leave_minutes_enough'] = summed_df.apply(leave_cal_minute_result, axis=1)
summed_df['leave_enough'] = summed_df.apply(lambda x: str(x['leave_day_enough']) + ':' + str(x['leave_hours_enough']) + ':' + str(x['leave_minutes_enough']), axis=1)
# summed_df['leave_day_enough'] = ((summed_df['leave_allow'] * 480) - summed_df['total_leave_data_minute']) // 480
# summed_df['leave_hours_enough'] = (((summed_df['leave_allow'] * 480) - summed_df['total_leave_data_minute']) - summed_df['leave_day_enough'] * 480) // 60
# summed_df['leave_minutes_enough'] = ((summed_df['leave_allow'] * 480) - summed_df['total_leave_data_minute']) - summed_df['leave_day_enough'] * 480 - summed_df['leave_hours_enough'] *60
result_df = summed_df.loc[:, ['full_name', 'startworkdate', 'durationwork','leave_type_name','leave_allow','leave_used','leave_enough']]
# summed_df.to_csv(f'output{year}', index=False)
# result_df.to_csv(f'result{year}', index=False)

/var/folders/kg/w853qbnj03x0b8wb35_f_v540000gp/T/ipykernel_37543/979193008.py:67: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  summed_df = df.groupby(['employee_id', 'full_name', 'leave_type_name','year','startworkdate']).sum().reset_index().sort_values(by='full_name')


In [260]:
# Melt the DataFrame for leave_allow
leave_allow_df = result_df.loc[:, ['full_name', 'startworkdate', 'durationwork', 'leave_type_name', 'leave_allow']]
leave_allow_df['leave_type'] = 'leave_allow'
leave_allow_df = leave_allow_df.rename(columns={'leave_allow': 'value'})
leave_allow_df = pd.melt(leave_allow_df, id_vars=['full_name', 'startworkdate', 'durationwork', 'leave_type_name', 'leave_type'], value_vars=['value'])

# Melt the DataFrame for leave_used
leave_used_df = result_df.loc[:, ['full_name', 'startworkdate', 'durationwork', 'leave_type_name', 'leave_used']]
leave_used_df['leave_type'] = 'leave_used'
leave_used_df = leave_used_df.rename(columns={'leave_used': 'value'})
leave_used_df = pd.melt(leave_used_df, id_vars=['full_name', 'startworkdate', 'durationwork', 'leave_type_name', 'leave_type'], value_vars=['value'])

# Melt the DataFrame for leave_enough
leave_enough_df = result_df.loc[:, ['full_name', 'startworkdate', 'durationwork', 'leave_type_name', 'leave_enough']]
leave_enough_df['leave_type'] = 'leave_enough'
leave_enough_df = leave_enough_df.rename(columns={'leave_enough': 'value'})
leave_enough_df = pd.melt(leave_enough_df, id_vars=['full_name', 'startworkdate', 'durationwork', 'leave_type_name', 'leave_type'], value_vars=['value'])

# Concatenate the DataFrames
result_long_df = pd.concat([leave_allow_df, leave_used_df, leave_enough_df])
result_long_df
# result_long_df.to_csv(f'result{year}.csv', index=False)

/var/folders/kg/w853qbnj03x0b8wb35_f_v540000gp/T/ipykernel_37543/417410714.py:5: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  leave_allow_df = pd.melt(leave_allow_df, id_vars=['full_name', 'startworkdate', 'durationwork', 'leave_type_name', 'leave_type'], value_vars=['value'])
/var/folders/kg/w853qbnj03x0b8wb35_f_v540000gp/T/ipykernel_37543/417410714.py:11: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  leave_used_df = pd.melt(leave_used_df, id_vars=['full_name', 'startworkdate', 'durationwork', 'leave_type_name', 'leave_type'], value_vars=['value'])
/var/folders/kg/w853qbnj03x0b8wb35_f_v540000gp/T/ipykernel_37543/41

,full_name,startworkdate,durationwork,leave_type_name,leave_type,variable,value
0,ธัณย์สิตา อริยภัคพรสิน,2017-08-01,2343 days,ลาคลอด หักค่าจ้าง,leave_allow,value,53
1,ธัณย์สิตา อริยภัคพรสิน,2017-08-01,2343 days,ลาพักร้อน,leave_allow,value,12
2,สรัลรัตน์ พึ่งละออ,2014-04-01,3561 days,ลาพักร้อน,leave_allow,value,15
3,สุวิมล สามเสน,2018-05-14,2057 days,ลาป่วย (ไม่หักค่าจ้าง),leave_allow,value,30
4,สุวิมล สามเสน,2018-05-14,2057 days,ลาพักร้อน,leave_allow,value,12
0,ธัณย์สิตา อริยภัคพรสิน,2017-08-01,2343 days,ลาคลอด หักค่าจ้าง,leave_used,value,10:0:0
1,ธัณย์สิตา อริยภัคพรสิน,2017-08-01,2343 days,ลาพักร้อน,leave_used,value,12:0:0
2,สรัลรัตน์ พึ่งละออ,2014-04-01,3561 days,ลาพักร้อน,leave_used,value,10:3:0
3,สุวิมล สามเสน,2018-05-14,2057 days,ลาป่วย (ไม่หักค่าจ้าง),leave_used,value,5:4:0
4,สุวิมล สามเสน,2018-05-14,2057 days,ลาพักร้อน,leave_used,value,9:0:0


In [257]:
pivot_df = pd.pivot_table(result_long_df, 
                          index=['full_name', 'startworkdate', 'durationwork'], 
                          columns=['leave_type_name','leave_type'],
                          values=['value'], 
                          fill_value=0,
                          aggfunc='sum')
pivot_df.to_excel(f'Result{year}.xlsx', index=True)

In [ ]:
# summed_df['end_of_year'] = pd.to_datetime(summed_df['year'], format='%Y') + pd.offsets.YearEnd(0)
# summed_df['startworkdate'] = pd.to_datetime(summed_df['startworkdate'], format='%Y-%m-%d')
# summed_df['durationwork'] = summed_df['end_of_year'] - summed_df['startworkdate']


In [ ]:
# summed_df

,employee_id,full_name,leave_type_name,year,startworkdate,lev_data_id,lev_data_d_day,lev_data_d_hour,lev_data_d_minute,leave_type,...,leave_day,leave_hours,leave_minutes,end_of_year,durationwork,durationwork_years,leave_allow,leave_day_enough,leave_hours_enough,leave_minutes_enough
620,202211010851451B,กชกร ช่างต่อ,ลาป่วย (ไม่หักค่าจ้าง),2022,2022-11-01,90899,2,3,0,99,...,2,3,0,2022-12-31,60 days,0.0,30,27,5,0
597,20220630092210sC,กชนันท์ ต่ายทรัพย์,ลากิจ (ไม่หักค่าจ้าง),2022,2022-07-01,83520,3,0,0,405,...,3,0,0,2022-12-31,183 days,0.0,3,0,0,0
598,20220630092210sC,กชนันท์ ต่ายทรัพย์,ลาป่วย (ไม่หักค่าจ้าง),2022,2022-07-01,58286,2,0,0,66,...,2,0,0,2022-12-31,183 days,0.0,30,28,0,0
288,2018071216000272,กนกวรรณ กล่ำนาค,ลาพักร้อน,2022,1997-12-03,205699,5,21,0,280,...,7,5,0,2022-12-31,9159 days,25.0,15,7,3,0
287,2018071216000272,กนกวรรณ กล่ำนาค,ลาป่วย (ไม่หักค่าจ้าง),2022,1997-12-03,73165,4,0,0,99,...,4,0,0,2022-12-31,9159 days,25.0,30,26,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,2018071216000291,แก่นนภา ศรีสง่า,ลาพักร้อน,2022,2007-03-01,120772,4,0,0,140,...,4,0,0,2022-12-31,5784 days,15.0,15,11,0,0
223,2018071216000229,แพรวพรรณ เหมทานนท์,ลาพักร้อน,2022,2013-09-16,291274,14,6,0,385,...,14,6,0,2022-12-31,3393 days,9.0,15,0,2,0
621,20221101085708vH,โฆสิต ทับอินทร,ลาป่วย (ไม่หักค่าจ้าง),2022,2022-11-01,122127,3,4,0,132,...,3,4,0,2022-12-31,60 days,0.0,30,26,4,0
607,2022080303234259,โชคชัย ปานเกลียว,ลากิจ (ไม่หักค่าจ้าง),2022,2022-08-08,30794,1,0,0,135,...,1,0,0,2022-12-31,145 days,0.0,3,2,0,0


In [ ]:
# df.loc[(df['full_name'] == 'วารุณี ผิวคาม')& (df['leave_type_name']=='ลาพักร้อน')]

,lev_data_id,lev_data_date_start,lev_data_date_end,lev_data_d_day,lev_data_d_hour,lev_data_d_minute,leave_type,detail,lev_approve_status,employee_id,...,updateBy,is_urgent,is_fix,full_name,leave_type_name,dep_name,brancheName,startworkdate,total_leave_data_minute,year
4494,20623,2022-01-04 08:00:00,2022-01-04 17:00:00,1,0,0,35,พักร้อน,1,2018071216000331,...,2018071216000318,1,0,วารุณี ผิวคาม,ลาพักร้อน,SQ,แผนก SQ บริหารบุคลากรลงไซต์งาน,2017-03-01,480,2022
4496,22781,2022-03-18 08:00:00,2022-03-18 17:00:00,1,0,0,35,ลาพักร้อน,1,2018071216000331,...,2018071216000318,1,0,วารุณี ผิวคาม,ลาพักร้อน,SQ,แผนก SQ บริหารบุคลากรลงไซต์งาน,2017-03-01,480,2022
4653,21695,2022-02-17 08:00:00,2022-02-17 17:00:00,1,0,0,35,ลาพักร้อน,1,2018071216000331,...,2018071216000318,1,0,วารุณี ผิวคาม,ลาพักร้อน,SQ,แผนก SQ บริหารบุคลากรลงไซต์งาน,2017-03-01,480,2022
4656,22783,2022-03-28 08:00:00,2022-03-30 17:00:00,3,0,0,35,ลาพักร้อน,1,2018071216000331,...,2018071216000318,1,0,วารุณี ผิวคาม,ลาพักร้อน,SQ,แผนก SQ บริหารบุคลากรลงไซต์งาน,2017-03-01,1440,2022
4777,21359,2022-02-02 08:00:00,2022-02-02 17:00:00,1,0,0,35,ลาพักร้อน,1,2018071216000331,...,2018071216000318,1,0,วารุณี ผิวคาม,ลาพักร้อน,SQ,แผนก SQ บริหารบุคลากรลงไซต์งาน,2017-03-01,480,2022
4781,22782,2022-03-21 08:00:00,2022-03-25 17:00:00,5,0,0,35,ลาพักร้อน,1,2018071216000331,...,2018071216000318,1,0,วารุณี ผิวคาม,ลาพักร้อน,SQ,แผนก SQ บริหารบุคลากรลงไซต์งาน,2017-03-01,2400,2022


In [ ]:
# summed_df.loc[(summed_df['full_name'] == 'วารุณี ผิวคาม')& (summed_df['leave_type_name']=='ลาพักร้อน')]

,employee_id,full_name,leave_type_name,year,startworkdate,lev_data_id,lev_data_d_day,lev_data_d_hour,lev_data_d_minute,leave_type,...,site_id,ot_id,lev_id,createBy,is_urgent,is_fix,total_leave_data_minute,leave_day,leave_hours,leave_minutes
372,2018071216000331,วารุณี ผิวคาม,ลาพักร้อน,2022,2015-03-02,132023,12,0,0,210,...,54,642.0,30,0.0,6,0,5760,12,0,0
